In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install --upgrade tensorflow
!pip install keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 84.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompat

In [4]:
import numpy as np
import math
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import json
import re

In [39]:
#mediapipeのデータを正規化
def readMedipipeToRow(json_file):
  data = json.load(json_file)
  #ランドマークの座標を一列の配列に変換
  coordinates = []
  max = -1
  x1 = 0.0
  y1 = 0.0
  z1 = 0.0
  for landmark in data[0]['landmarks']:
    print(landmark)
    if landmark['name'] == 'WRIST':
      x1 = landmark['x']
      y1 = landmark['y']
      z1 = landmark['z']
    else:
      t = pow(landmark['x']-x1, 2) + pow(landmark['y']-y1, 2) + pow(landmark['z']-z1, 2)
      if t > max: max = t
      print(max)
      new_landmark = [landmark['x'] - x1, landmark['y'] - y1, landmark['z'] - z1]  # (1, 3) の形にする
      # axis=0 で新しい行を追加
      print(new_landmark)
      coordinates.append(new_landmark)
  return coordinates/math.sqrt(max)

In [40]:
#正規化したデータを配列に変換
directory = '/content/drive/Shareddrives/JPHackathon-2-241027/単語手話/annotated_points_flip'
x = []
y = []
# 指定したディレクトリ内の全てのサブディレクトリを取得
subdirs = sorted(
    [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))],
    key=lambda x: int(x) if x.isdigit() else x
)

# 各サブディレクトリを一つずつ処理
for subdir in subdirs:
    subdir_path = os.path.join(directory, subdir)
    print(f"Processing directory: {subdir_path}")

    # サブディレクトリ内のファイルを取得
    for file in os.listdir(subdir_path):
        # JSONファイルを探す
        if file.endswith('.json'):
            file_path = os.path.join(subdir_path, file)
            try:
                # JSONファイルを読み込み、内容を出力
                with open(file_path, 'r', encoding='utf-8') as json_file:
                  data = json.load(json_file)
                  #ランドマークの座標を一列の配列に変換
                  coordinates = []
                  max = -1
                  x1 = 0.0
                  y1 = 0.0
                  z1 = 0.0
                  for landmark in data[0]['landmarks']:
                    if landmark['name'] == 'WRIST':
                      x1 = landmark['x']
                      y1 = landmark['y']
                      z1 = landmark['z']
                    else:
                      t = pow(landmark['x']-x1, 2) + pow(landmark['y']-y1, 2) + pow(landmark['z']-z1, 2)
                      if t > max: max = t
                      new_landmark = [landmark['x'] - x1, landmark['y'] - y1, landmark['z'] - z1]  # (1, 3) の形にする
                      # axis=0 で新しい行を追加
                      coordinates = np.append(coordinates, new_landmark)
                  coordinates = np.array(coordinates) / math.sqrt(max)
                  x.append(coordinates)
                  #x.append(readMedipipeToRow(json_file))
                  match = re.match(r"(\d+)-[a-z]+", subdir)
                  if match:
                    number = match.group(1)
                    number = int(number)
                  y.append(number)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

# NumPy配列に変換
x = np.array(x)
y = np.array(y)

print(x)
print(y)

# データの分割
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


Processing directory: /content/drive/Shareddrives/JPHackathon-2-241027/単語手話/annotated_points_flip/00-a
Processing directory: /content/drive/Shareddrives/JPHackathon-2-241027/単語手話/annotated_points_flip/01-i
Processing directory: /content/drive/Shareddrives/JPHackathon-2-241027/単語手話/annotated_points_flip/02-u
Processing directory: /content/drive/Shareddrives/JPHackathon-2-241027/単語手話/annotated_points_flip/03-e
Processing directory: /content/drive/Shareddrives/JPHackathon-2-241027/単語手話/annotated_points_flip/04-o
Processing directory: /content/drive/Shareddrives/JPHackathon-2-241027/単語手話/annotated_points_flip/05-ka
Processing directory: /content/drive/Shareddrives/JPHackathon-2-241027/単語手話/annotated_points_flip/06-ki
Processing directory: /content/drive/Shareddrives/JPHackathon-2-241027/単語手話/annotated_points_flip/07-ku
Processing directory: /content/drive/Shareddrives/JPHackathon-2-241027/単語手話/annotated_points_flip/08-ke
Processing directory: /content/drive/Shareddrives/JPHackathon-2-24102

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler

# データの標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# モデルの構築
model = Sequential([
    Dense(128, activation='relu', input_shape=(40,)),
    Dense(64, activation='relu'),
    Dense(len(set(y)), activation='softmax')
])

# モデルのコンパイル
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# モデルの訓練
model.fit(X_train, y_train, epochs=100, batch_size=32)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.1771 - loss: 2.5309
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4147 - loss: 1.8966  
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5765 - loss: 1.5836  
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7004 - loss: 1.3399
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7703 - loss: 1.0939
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8225 - loss: 0.8703
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8683 - loss: 0.7013
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9064 - loss: 0.5523  
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9511 - loss: 0.4734
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9523 - loss: 0.3713
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9312 - loss: 0.3382  
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - 

In [42]:
# モデルの評価
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9532 - loss: 0.2889
Test accuracy: 0.9610389471054077


In [43]:
# 新しいデータに対する予測
predictions = model.predict(X_test)
print(predictions[0][1])
predicted_classes = np.argmax(predictions, axis=1)
print(f'Predicted classes: {predicted_classes}')
print(y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
4.986179e-08
Predicted classes: [11  9  8 12  2 14  1  4 14  9 14  4  7 11  4  6  6 14 14  7  1  6  0  3
  0  2  5  9  0  9  4  0 12  3  5 14  7  1 13 11  3  4  4  6 14  0  1  3
  3  2  1  3 12 11  5 11  8  3  0  7  1  9  3 10 11  8 13  5 14  0  0  5
  4  6  2  9  3]
[11  9  8 12  2 14  1  4 14  9 14  4  7 11  4  6  6 14 14  9  1  1  0  3
  0  2  5  9  0  9  4  0 12  3  5 14  7  1 13 11  3  4  4  6  5  0  1  3
  3  2  1  3 12 11  5 11  8  3  0  7  1  9  3 10 11  8 13  5 14  0  0  5
  4  6  2  9  3]


Tensorflow liteを保存して実行する

In [44]:
folder = '/content/drive/Shareddrives/JPHackathon-2-241027/単語手話/classify hands/'
model.save(folder + 'a-so-ikami-model-flip-xy.keras')

# Keras モデルを TensorFlow Lite モデルに変換
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # 直接モデルを指定
tflite_model = converter.convert()

open(folder + "a-so-ikami-model-flip-xy.tflite", "wb").write(tflite_model)

Saved artifact at '/tmp/tmpp97xywz4'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 40), dtype=tf.float32, name='keras_tensor_12')
Output Type:
  TensorSpec(shape=(None, 15), dtype=tf.float32, name=None)
Captures:
  136086435648384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136086436917520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136086436916816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136086436915056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136086436919104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136086436918752: TensorSpec(shape=(), dtype=tf.resource, name=None)


60072

In [31]:
# TFLiteモデルの読み込み
interpreter = tf.lite.Interpreter(model_path= folder + "a-so-ikami-mikubo-model-flip.tflite")
# メモリ確保。これはモデル読み込み直後に必須
interpreter.allocate_tensors()
# 学習モデルの入力層・出力層のプロパティをGet.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [21]:
print(input_details[0]['shape'])

[ 1 60]


In [22]:
print(input_details[0]['index'])

0


In [32]:
# X_testからデータを取得し、型をfloat32に変換
input_data = X_test[0].astype(np.float32).reshape(input_details[0]['shape'])  # リシェイプ
interpreter.set_tensor(input_details[0]['index'], input_data)

# 推論の実行
interpreter.invoke()

# 出力データの取得
output_data = interpreter.get_tensor(output_details[0]['index'])

# 結果の表示
print(f"Output for test sample {0}:", output_data)

Output for test sample 0: [[3.9319362e-05 9.9671197e-01 7.5347803e-06 1.2513435e-04 2.5114680e-06
  4.5927845e-05 4.4909135e-05 3.7860852e-06 2.3088618e-03 5.5622459e-06
  4.3334574e-05 2.2004529e-06 1.1699615e-07 6.4005307e-04 1.8635941e-05]]


In [33]:
print(y_test[0])

1
